In [ ]:
!pip install xlsxwriter

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

# Load the Excel file
file_path = '/content/final_lagged.xlsx'  # Specify your file path
xls = pd.ExcelFile(file_path)

# Create a new Excel writer object
with pd.ExcelWriter('knn_lagged.xlsx', engine='xlsxwriter') as writer:

    # Iterate over all sheets
    for sheet_name in xls.sheet_names:
        # Read each sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Create a mask for rows where all values are missing (excluding 'Timestamp')
        if 'Timestamp' in df.columns:
            df_no_timestamp = df.drop(columns=['Timestamp'])
        else:
            df_no_timestamp = df

        all_missing_mask = df_no_timestamp.isnull().all(axis=1)

        # Apply KNN imputation to rows with partial missing values
        if df_no_timestamp.isnull().values.any():
            # Create a copy for KNN imputation
            df_knn = df_no_timestamp.copy()

            # Apply KNN imputation
            imputer = KNNImputer(n_neighbors=5)
            df_knn_imputed = pd.DataFrame(imputer.fit_transform(df_knn), columns=df_knn.columns, index=df.index)

            # Restore rows where all values were missing
            df_knn_imputed[all_missing_mask] = df_no_timestamp[all_missing_mask]

            # Reinsert the 'Timestamp' column back into the DataFrame
            if 'Timestamp' in df.columns:
                df_knn_imputed.insert(0, 'Timestamp', df['Timestamp'])

            # Convert 'Timestamp' back to datetime format
            if 'Timestamp' in df_knn_imputed.columns:
                df_knn_imputed['Timestamp'] = pd.to_datetime(df['Timestamp'])

            # Write the KNN imputed DataFrame back to the Excel writer
            df_knn_imputed.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            # If no NaN values are present, just write the original DataFrame
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save the output
    writer.close()

print("KNN imputation applied for partial missing data. Fully missing rows left unchanged. Output saved to 'knn_interpolated_output.xlsx'")


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/knn_lagged.xlsx'  # Specify your file path
xls = pd.ExcelFile(file_path)

# Create a new Excel writer object
with pd.ExcelWriter('imputed_lagged.xlsx', engine='xlsxwriter') as writer:

    # Iterate over all sheets
    for sheet_name in xls.sheet_names:
        # Read each sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Apply linear interpolation for missing values
        df_interpolated = df.interpolate(method='linear', limit_direction='both')

        # Write the interpolated DataFrame back to the Excel writer
        df_interpolated.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save the output
    writer.close()

print("Linear interpolation applied and saved to 'interpolated_output.xlsx'")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the Excel file
file_path ='/content/Correlations(pm10_including zero).xlsx' # Replace with your file path

# List of seasons for labeling
seasons = ['corr_summer', 'corr_monsoon', 'corr_post_mons', 'corr_winter']

# Read all sheets into a dictionary of DataFrames
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Create a box plot for each sheet (variable)
for variable, df in sheets_dict.items():
    # Assuming the DataFrame has stations as rows and seasons as columns
    data_to_plot = df[seasons]

    # Create a box plot for the current variable
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=data_to_plot)

    # Set the title and labels
    plt.title(f'Box Plot for {variable} for 2023', fontsize=16)
    plt.xlabel('Season', fontsize=12)
    plt.ylabel('Correlation with PM10', fontsize=12)

    # Save the plot or show it
    plt.savefig(f'{variable}_boxplot(2023)_new.png')  # Saves the plot
    plt.show()  # To display the plot


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/imputed_all_cites(23)_new.xlsx'  # Update this with the actual file path
excel_file = pd.ExcelFile(file_path)

# Define the timestamp range
start_timestamp = '2023-10-01'  # Start of the range
end_timestamp = '2023-11-30'  # End of the range

# Initialize a dictionary to store normalized data
normalized_sheets = {}

# Iterate through each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    # Read the current sheet into a DataFrame
    df = excel_file.parse(sheet_name)

    # Convert 'Timestamp' column to datetime format
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Filter data for the specific timestamp range
    df_filtered = df[(df['Timestamp'] >= start_timestamp) & (df['Timestamp'] <= end_timestamp)]

    # Remove the 'PM10' column
    df_filtered = df_filtered.drop(columns=['PM2.5 (µg/m³)'], errors='ignore')

    # Define variables to normalize (all remaining columns except 'Timestamp')
    variables_to_normalize = df_filtered.columns[1:]  # Exclude 'Timestamp'

    # Normalize each variable (PM2.5 and the other 9 variables) within the filtered time range
    df_normalized = df_filtered.copy()
    df_normalized[variables_to_normalize] = df_filtered[variables_to_normalize].apply(
        lambda x: (x - x.mean()) / x.std()
    )

    # Store the normalized DataFrame in the dictionary
    normalized_sheets[sheet_name] = df_normalized

# Save the normalized sheets back into a new Excel file
output_file_path = 'normalized_post_monsoon(pm10)_new.xlsx'  # Update with your desired output file path
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df_normalized in normalized_sheets.items():
        df_normalized.to_excel(writer, sheet_name=sheet_name, index=False)

print("Normalization complete for the specified timestamp range. The file has been saved as:", output_file_path)

In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/imputed_all_cites(23)_new.xlsx'  # Update this with the actual file path
excel_file = pd.ExcelFile(file_path)

# Define the timestamp ranges
timestamp_ranges = [
    ('2023-12-01', '2023-12-31'),
    ('2023-01-01', '2023-02-28')
]

# Initialize a dictionary to store normalized data
normalized_sheets = {}

# Iterate through each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    # Read the current sheet into a DataFrame
    df = excel_file.parse(sheet_name)

    # Convert 'Timestamp' column to datetime format
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Filter data for the specific timestamp ranges
    df_filtered = pd.concat([
        df[(df['Timestamp'] >= start) & (df['Timestamp'] <= end)]
        for start, end in timestamp_ranges
    ])

    # Remove the 'PM10' column
    df_filtered = df_filtered.drop(columns=['PM2.5 (µg/m³)'], errors='ignore')

    # Define variables to normalize (all remaining columns except 'Timestamp')
    variables_to_normalize = df_filtered.columns[1:]  # Exclude 'Timestamp'

    # Normalize each variable (PM2.5 and the other 9 variables) within the filtered time range
    df_normalized = df_filtered.copy()
    df_normalized[variables_to_normalize] = df_filtered[variables_to_normalize].apply(
        lambda x: (x - x.mean()) / x.std()
    )

    # Store the normalized DataFrame in the dictionary
    normalized_sheets[sheet_name] = df_normalized

# Save the normalized sheets back into a new Excel file
output_file_path = 'normalized_winter(pm10)_new.xlsx'  # Update with your desired output file path
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df_normalized in normalized_sheets.items():
        df_normalized.to_excel(writer, sheet_name=sheet_name, index=False)

print("Normalization complete for the specified timestamp ranges. The file has been saved as:", output_file_path)

In [ ]:
import pandas as pd
import numpy as np

# Load the Excel file with multiple sheets
file_path = '/content/normalized_post_monsoon(pm10)_new.xlsx'  # Replace with your file path
excel_data = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary

# Define the post-monsoon period
start_date = '2023-10-01'
end_date = '2023-11-30'

# Variables of interest (independent variables and PM2.5)
independent_vars = ['NO2 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)', 'Ozone (µg/m³)', 'AT (°C)', 'RH (%)', 'WS (m/s)', 'WD (deg)']
dependent_var = 'PM10 (µg/m³)'

# Initialize dictionaries to store individual sheet data
R_xx_dict = {}
R_xy_dict = {}
result_dict = {}

# Loop through each sheet in the Excel file
for sheet_name, df in excel_data.items():
    # Ensure the timestamp is in datetime format
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Filter the data based on the post-monsoon period
    post_monsoon_data = df[(df['Timestamp'] >= start_date) & (df['Timestamp'] <= end_date)]

    # Collect the independent variables and dependent variable data
    independent_data = post_monsoon_data[independent_vars]
    dependent_data = post_monsoon_data[dependent_var]

    # Calculate correlation matrix R_xx for the independent variables
    R_xx = independent_data.corr()
    R_xx_dict[sheet_name] = R_xx  # Store R_xx in dictionary

    # Calculate correlation matrix R_xy between PM2.5 and independent variables
    R_xy = independent_data.corrwith(dependent_data)
    R_xy = pd.DataFrame(R_xy, columns=[dependent_var])
    R_xy_dict[sheet_name] = R_xy  # Store R_xy in dictionary

    # Calculate the inverse of R_xx
    R_xx_inv = np.linalg.inv(R_xx)

    # Multiply R_xx inverse with R_xy
    result = pd.DataFrame(R_xx_inv @ R_xy, columns=[dependent_var])
    result_dict[sheet_name] = result  # Store result in dictionary

# Write R_xx to one Excel file with multiple sheets
with pd.ExcelWriter('R_xx_post_monsoon(pm10)_new.xlsx') as writer:
    for sheet_name, R_xx in R_xx_dict.items():
        R_xx.to_excel(writer, sheet_name=sheet_name)

# Write R_xy to another Excel file with multiple sheets
with pd.ExcelWriter('R_xy_post_monsoon(pm10)_new.xlsx') as writer:
    for sheet_name, R_xy in R_xy_dict.items():
        R_xy.to_excel(writer, sheet_name=sheet_name)

# Write R_xx_inv * R_xy to a third Excel file with multiple sheets
with pd.ExcelWriter('R_xx_inv_times_R_xy_post_monsoon(pm10)_new.xlsx') as writer:
    for sheet_name, result in result_dict.items():
        result.to_excel(writer, sheet_name=sheet_name)

print("Results saved to separate Excel files successfully!")


In [ ]:
import pandas as pd
import numpy as np

# Load the Excel file with multiple sheets
file_path = '/content/normalized_winter(pm10)_new.xlsx'  # Replace with your file path
excel_data = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary

# Define the winter periods
winter_period1_start = '2023-12-01'
winter_period1_end = '2023-12-31'
winter_period2_start = '2023-01-01'
winter_period2_end = '2023-02-28'

# Variables of interest (independent variables and PM2.5)
independent_vars = ['NO2 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)', 'Ozone (µg/m³)', 'AT (°C)', 'RH (%)', 'WS (m/s)', 'WD (deg)']
dependent_var = 'PM10 (µg/m³)'

# Initialize dictionaries to store individual sheet data
R_xx_dict = {}
R_xy_dict = {}
result_dict = {}

# Loop through each sheet in the Excel file
for sheet_name, df in excel_data.items():
    # Ensure the timestamp is in datetime format
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Filter the data based on the two winter periods
    winter_data = df[
        ((df['Timestamp'] >= winter_period1_start) & (df['Timestamp'] <= winter_period1_end)) |
        ((df['Timestamp'] >= winter_period2_start) & (df['Timestamp'] <= winter_period2_end))
    ]

    # Collect the independent variables and dependent variable data
    independent_data = winter_data[independent_vars]
    dependent_data = winter_data[dependent_var]

    # Calculate correlation matrix R_xx for the independent variables
    R_xx = independent_data.corr()
    R_xx_dict[sheet_name] = R_xx  # Store R_xx in dictionary

    # Calculate correlation matrix R_xy between PM2.5 and independent variables
    R_xy = independent_data.corrwith(dependent_data)
    R_xy = pd.DataFrame(R_xy, columns=[dependent_var])
    R_xy_dict[sheet_name] = R_xy  # Store R_xy in dictionary

    # Calculate the inverse of R_xx
    R_xx_inv = np.linalg.inv(R_xx)

    # Multiply R_xx inverse with R_xy
    result = pd.DataFrame(R_xx_inv @ R_xy, columns=[dependent_var])
    result_dict[sheet_name] = result  # Store result in dictionary

# Write R_xx to one Excel file with multiple sheets
with pd.ExcelWriter('R_xx_winter(pm10)_new.xlsx') as writer:
    for sheet_name, R_xx in R_xx_dict.items():
        R_xx.to_excel(writer, sheet_name=sheet_name)

# Write R_xy to another Excel file with multiple sheets
with pd.ExcelWriter('R_xy_winter(pm10)_new.xlsx') as writer:
    for sheet_name, R_xy in R_xy_dict.items():
        R_xy.to_excel(writer, sheet_name=sheet_name)

# Write R_xx_inv * R_xy to a third Excel file with multiple sheets
with pd.ExcelWriter('R_xx_inv_times_R_xy_winter(pm10)_new.xlsx') as writer:
    for sheet_name, result in result_dict.items():
        result.to_excel(writer, sheet_name=sheet_name)

print("Results saved to separate Excel files successfully!")


In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Load the Excel file
file_path = '/content/imputed_all_cites(23)_new.xlsx'
excel_data = pd.ExcelFile(file_path)
sheet_names = excel_data.sheet_names

# Initialize an output dictionary to store VIF results
vif_results = {}

# Define the date range
start_date = '2023-03-01'
end_date = '2023-05-31'

for sheet in sheet_names:
    # Load the sheet
    data = excel_data.parse(sheet)

    # Ensure the timestamp column is in datetime format
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])

    # Filter the data for the specific date range
    filtered_data = data[(data['Timestamp'] >= start_date) & (data['Timestamp'] <= end_date)]

    # Drop timestamp and PM10 columns
    filtered_data = filtered_data.drop(columns=['Timestamp', 'PM10 (µg/m³)'])

    # Separate PM2.5 as the dependent variable
    dependent_var = 'PM2.5 (µg/m³)'
    filtered_data = filtered_data.drop(columns=[dependent_var])

    # Add a constant to the data for VIF calculation
    X = add_constant(filtered_data)

    # Calculate VIF for each variable
    vif_data = pd.DataFrame()
    vif_data['Variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # Store the VIF results in the output dictionary
    vif_results[sheet] = vif_data

# Save the results to an Excel file
output_path = 'vif_results_summer.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for sheet, vif_data in vif_results.items():
        vif_data.to_excel(writer, sheet_name=sheet, index=False)

print(f"VIF results saved to {output_path}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Create a DataFrame with your data
data = {
        'Factor': ['NO2', 'SO2', 'CO', 'AT', 'RH', 'WS'],
        'Direct': [45, None, 43, 31, None, 19],
        'Indirect': [17, None, 8, 33, None, 44],
        'Both': [26, None, 43, 17, None, 44],
        'No effect': [11, None,5, 19, None, 20],
        'Correlation': ['+ve', 'zero', '+ve', '-ve', 'zero', '-ve']
}

df = pd.DataFrame(data)

# 2. Reshape the data from wide to long form for plotting
df_melt = df.melt(
    id_vars=['Factor','Correlation'],
    value_vars=['Direct','Indirect','Both','No effect'],
    var_name='Category',
    value_name='Value'
)

# 3. Map correlation signs to colors
color_map = {
    '+ve':  'blue',
    '-ve':  'red',
    'zero': 'gray'
}

# 4. Define the order for categories & factors (to match your layout)
categories = ['Direct','Indirect','Both','No effect']
factors    = ['NO2','SO2','CO','AT','WS','RH']

# 5. Convert categories and factors to numeric positions
x_positions = {cat: i for i, cat in enumerate(categories)}
y_positions = {fac: i for i, fac in enumerate(factors)}

# 6. Create the plot
fig, ax = plt.subplots(figsize=(8,5))

# Enable grid lines behind the scatter points
ax.set_axisbelow(True)
ax.xaxis.grid(True, color='lightgray', linewidth=0.5)
ax.yaxis.grid(True, color='lightgray', linewidth=0.5)

# 7. Plot bubbles or dashes
for _, row in df_melt.iterrows():
    cat = row['Category']
    fac = row['Factor']
    val = row['Value']
    cor = row['Correlation']

    x_val = x_positions[cat]
    y_val = y_positions[fac]

    # If value is missing or zero, show a dash
    if pd.isnull(val) or val == 0:
        ax.text(
            x_val, y_val,
            '-',
            ha='center', va='center',
            fontsize=12
        )
        continue

    # Otherwise, draw a bubble
    bubble_size  = val * 30  # Scale circle size by the percentage
    bubble_color = color_map.get(cor, 'gray')

    scatter = ax.scatter(
        x_val, y_val,
        s=bubble_size,
        c=bubble_color,
        alpha=0.6,
        edgecolors='k'
    )

    # Convert the value to integer to avoid floats, then add a "%" sign
    ax.text(
        x_val, y_val,
        f"{int(val)}%",
        ha='center',
        va='center',
        fontsize=6.5,
        color='white',    # or 'black', depending on your preference
        weight='bold'     # make it bold if desired
    )

# 8. Configure axis ticks and labels
ax.set_xticks(range(len(categories)))
ax.set_xticklabels(categories)
ax.set_yticks(range(len(factors)))
ax.set_yticklabels(factors)

ax.set_xlim(-0.5, len(categories) - 0.5)
ax.set_ylim(-0.5, len(factors) - 0.5)

ax.set_xlabel('Category')
ax.set_ylabel('Factor')
ax.set_title('Winter_PM10')

# 9. Place the legend outside the main plotting area
for corr_label, color in color_map.items():
    ax.scatter([], [], c=color, alpha=0.6, edgecolors='k', label=corr_label)
ax.legend(
    title='Correlation',
    loc='center left',       # anchor the legend to the left center of the bounding box
    bbox_to_anchor=(1, 0.5)  # place the bounding box to the right of the axes
)

# Ensure enough space on the right for the legend
plt.tight_layout(rect=[0, 0, 0.8, 1])
plt.show()